**Laboratorio N°3 - SIS420**

**Introducción.**

En este cuadernillo, para este laboratorio, nos enfocaremos en el entrenamiento y evaluación de un modelo de regresión logística para la clasificación de subclases de trastornos genéticos en pacientes. Utilizaremos un conjunto de datos específico que contiene variables médicas y genéticas como características independientes, y la subclase del trastorno genético como variable a predecir.

**Objetivos.**

- Seleccionar y preparar el dataset "Predict The Genetic Disorder" que contiene las variables necesarias para el entrenamiento de modelos de regresión logística.
- Implementar el modelo de regresión logística utilizando la estrategia one vs all para la clasificación de subclases de trastornos genéticos.
- Entrenar y validar cada modelo utilizando técnicas adecuadas de división de datos y validación cruzada.
- Evaluar la efectividad del modelo mediante la realización de predicciones y la medición de métricas de desempeño.
- Utilizar la biblioteca Pandas para manipular y procesar los datos del dataset.
- Generar gráficos de costo para analizar el proceso de entrenamiento de los modelos.


In [33]:
print(f"Laboratorio N°3 - SIS420: Gabriel Aparicio Llanquipacha.")
print(f"Carrera: Ingeniería en Ciencias de la Computación.")

Laboratorio N°3 - SIS420: Gabriel Aparicio Llanquipacha.
Carrera: Ingeniería en Ciencias de la Computación.


**(0) Montar Google Drive en Colaboratoy.** Una vez que el Google Drive está montado, puedes acceder a los archivos almacenados en tu Google Drive directamente desde el entorno de Colaboratory.

In [34]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


**(1) Preparación de datos y funciones para el modelo.**



**(a) Establecimiento de las Librerías.** En esta sección, importamos las librerías necesarias para nuestro análisis y entrenamiento del modelo. Las librerías incluyen pandas para manipulación de datos, numpy para cálculos numéricos, matplotlib para visualización de datos, scipy para optimización y sklearn para dividir el conjunto de datos en entrenamiento y prueba.

In [35]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy import optimize
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
%matplotlib inline

**(b) Creación de las funciones.** Se crean las funciones para que el código sea más legible e interpretable, además de que con este enfoque modular el código se presentará más limpio y se enfocará más en el algoritmo como tal, aquí se irán agregando las siguientes funciones:

*(i) Sigmoide.* La función calcularSigmoide calcula la función sigmoide, que transforma un valor lineal en un valor entre 0 y 1, que puede interpretarse como la probabilidad de que una instancia pertenezca a la clase positiva.

In [36]:
def calcularSigmoide(z):
    z = np.clip(z, -500, 500)
    return 1.0 / (1.0 + np.exp(-z))

*(ii) Función de Costo.* La función calcularCosto calcula la función de costo y el gradiente para la regresión logística con regularización. Esto se utiliza durante el entrenamiento del modelo para evaluar la precisión de las predicciones y ajustar los parámetros para minimizar el error.

In [37]:
def calcularCosto(theta, X, y, lambda_):
    m = y.size
    J = 0
    grad = np.zeros(theta.shape)
    h = calcularSigmoide(X.dot(theta.T))
    temp = theta.copy()
    temp[0] = 0
    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))
    grad = (1 / m) * (h - y).dot(X) + (lambda_ / m) * temp
    return J, grad

*(iii) Entrenamiento One vs All.* La función oneVsAll implementa la estrategia "one-vs-all" para entrenar un clasificador de regresión logística multinomial. Entrena un modelo separado para cada clase y devuelve los parámetros entrenados para cada clase.

In [38]:
def oneVsAll(X, y, num_labels, lambda_):
    m, n = X.shape
    all_theta = np.zeros((num_labels, n + 1))
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    for c in np.arange(num_labels):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 5000}
        res = optimize.minimize(calcularCosto,
                                initial_theta,
                                (X, (y == (c + 1)), lambda_),
                                jac=True,
                                method='BFGS',
                                options=options)
        all_theta[c] = res.x
    return all_theta

*(iv) Predicción One vs All.* La función predictOneVsAll realiza predicciones utilizando los parámetros entrenados por el enfoque "one-vs-all". Devuelve la clase con la probabilidad más alta para cada instancia de entrada.

In [39]:
def predictOneVsAll(all_theta, X):
    m = X.shape[0];
    num_labels = all_theta.shape[0]
    p = np.zeros(m)
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(calcularSigmoide(X.dot(all_theta.T)), axis = 1)
    return p + 1

**(c) Preprocesamiento de los datos.** Se procesarán y guardarán los datos en las respectivas variables para el entrenamiento, para esta parte se usará pandas.

*(i) Carga del dataset.* Cargamos el conjunto de datos desde el archivo CSV especificado.



In [40]:
dataset = pd.read_csv('/content/gdrive/MyDrive/SIS420/Laboratorio-N3_-_SIS420/Dataset/Predict The Genetic Disorder/Predict The Genetic Disorder.csv', sep=';', header=0, decimal=',')

*(ii) Extracción de datos existentes sin repetir.* Extraemos los valores únicos para cada columna de tipo objeto en el conjunto de datos. Esto nos ayudará a mapear los valores categóricos a números enteros.

In [41]:
datos = {}
columnas = dataset.columns[dataset.dtypes == 'object'].tolist()
for columna in columnas:
    datos[columna] = dataset[columna].drop_duplicates().values
print(datos)

{"Genes in mother's side": array(['Yes', 'No'], dtype=object), 'Inherited from father': array(['No', 'Yes', nan], dtype=object), 'Maternal gene': array(['Yes', 'No', nan], dtype=object), 'Paternal gene': array(['No', 'Yes'], dtype=object), 'Status': array(['Alive', 'Deceased'], dtype=object), 'Respiratory Rate (breaths/min)': array(['Normal (30-60)', 'Tachypnea', nan, '-99'], dtype=object), 'Heart Rate (rates/min': array(['Normal', 'Tachycardia', nan, '-99'], dtype=object), 'Parental consent': array(['Yes', nan, '-99'], dtype=object), 'Follow-up': array(['High', 'Low', nan, '-99'], dtype=object), 'Gender': array([nan, 'Male', 'Female', 'Ambiguous', '-99'], dtype=object), 'Birth asphyxia': array([nan, 'No', 'No record', 'Not available', 'Yes', '-99'],
      dtype=object), 'Autopsy shows birth defect (if applicable)': array(['Not applicable', 'None', 'No', 'Yes', nan, '-99'], dtype=object), 'Folic acid details (peri-conceptional)': array(['No', 'Yes', nan, '-99'], dtype=object), 'H/O ser

*(iii) Reemplazo de valores object a enteros.* Reemplazamos los valores categóricos en el conjunto de datos con números enteros utilizando el mapeo obtenido en el paso anterior.

In [42]:
datos_num = {}
for columna, valores in datos.items():
    indice_reemp = 1
    datos_num_col = {}
    for valor in valores:
        if valor not in datos_num_col and not pd.isnull(valor):
            datos_num_col[valor] = indice_reemp
            indice_reemp += 1
    if np.nan not in datos_num_col:
       datos_num_col[np.nan] = 1
    datos_num[columna] = datos_num_col
print(datos_num)

{"Genes in mother's side": {'Yes': 1, 'No': 2, nan: 1}, 'Inherited from father': {'No': 1, 'Yes': 2, nan: 1}, 'Maternal gene': {'Yes': 1, 'No': 2, nan: 1}, 'Paternal gene': {'No': 1, 'Yes': 2, nan: 1}, 'Status': {'Alive': 1, 'Deceased': 2, nan: 1}, 'Respiratory Rate (breaths/min)': {'Normal (30-60)': 1, 'Tachypnea': 2, '-99': 3, nan: 1}, 'Heart Rate (rates/min': {'Normal': 1, 'Tachycardia': 2, '-99': 3, nan: 1}, 'Parental consent': {'Yes': 1, '-99': 2, nan: 1}, 'Follow-up': {'High': 1, 'Low': 2, '-99': 3, nan: 1}, 'Gender': {'Male': 1, 'Female': 2, 'Ambiguous': 3, '-99': 4, nan: 1}, 'Birth asphyxia': {'No': 1, 'No record': 2, 'Not available': 3, 'Yes': 4, '-99': 5, nan: 1}, 'Autopsy shows birth defect (if applicable)': {'Not applicable': 1, 'None': 2, 'No': 3, 'Yes': 4, '-99': 5, nan: 1}, 'Folic acid details (peri-conceptional)': {'No': 1, 'Yes': 2, '-99': 3, nan: 1}, 'H/O serious maternal illness': {'Yes': 1, 'No': 2, '-99': 3, nan: 1}, 'H/O radiation exposure (x-ray)': {'No': 1, 'Not

*(iv) Reemplazo en el dataset* Aplicamos los cambios al conjunto de datos original.

In [43]:
for columna, d_n in datos_num.items():
    dataset[columna] = dataset[columna].replace(d_n)
dataset = dataset.fillna(0)
dataset.replace(0, 1, inplace=True)
columnas_a_modificar = ['Test 1', 'Test 2', 'Test 3', 'Test 4', 'Test 5']
dataset[columnas_a_modificar] = dataset[columnas_a_modificar].applymap(lambda x: x + 1)

*(v) División del Dataset para trabajo.* Dividimos el conjunto de datos en un conjunto de entrenamiento y un conjunto de prueba para evaluar el rendimiento del modelo.

In [44]:
train_set, test_set = train_test_split(dataset, test_size=0.2, random_state=101)
X_train, y_train = train_set.iloc[:, :-1], train_set.iloc[:, -1]
X_test, y_test = test_set.iloc[:, :-1], test_set.iloc[:, -1]
X_train = np.concatenate([np.ones((X_train.shape[0], 1)), X_train], axis=1)
X_test = np.concatenate([np.ones((X_test.shape[0], 1)), X_test], axis=1)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

**(2) Entrenamiento del Modelo One vs All.** Se entrenará al modelo con el dataset respectivo, esto de forma que se puedan realizar predicciones posteriores.

**(a) Calcular el Costo.** Calculamos el costo inicial y los gradientes antes de entrenar el modelo.

In [45]:
tamaño_capa_entrada = X_train.shape[1]
num_etiquetas = len(np.unique(y_train))
theta_t = np.zeros(X_train.shape[1])
lambda_t = 0.0001
J, grad = calcularCosto(theta_t, X_train, y_train, lambda_t)
print('Costo         : {:.6f}'.format(J))
print('-----------------------')
print('Gradientes:')
print(grad)

Costo         : 0.693147
-----------------------
Gradientes:
[ 0.          0.0832937  -0.2296507   0.15485586 -0.41932178  0.18425108
  0.03207266  0.37720236  0.33214187 -0.01376854  0.28187743  0.27717185
 -0.66148241 -0.64856842 -0.65712915 -0.65463208 -0.65400672  0.6454156
  0.44900973  0.41564929  0.35527467  0.18206482  0.43035889  0.4246054
  0.39278468  0.38568854  0.45970162  0.44760578 -0.64766107  0.44578324
 -0.63769592  0.65663961 -0.29913831  0.          0.          0.
  0.          0.24389943]


**(b) Obtención de los parámetros theta.** Entrenamos el modelo utilizando la estrategia "one-vs-all" y obtenemos los parámetros theta para cada clase.

In [46]:
theta = oneVsAll(X_train, y_train, num_etiquetas, lambda_t)
print(theta)

<ipython-input-37-9ab3398911a5>:8: RuntimeWarning: divide by zero encountered in log
  J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))
<ipython-input-37-9ab3398911a5>:8: RuntimeWarning: divide by zero encountered in log
  J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))


[[ 1.72974520e+00  0.00000000e+00  3.14970739e-02 -9.58438440e-02
   6.25963293e-02 -2.12247850e-01  8.74366205e-02  1.09978713e-02
  -3.76713694e-02 -5.66499642e-02 -4.77115349e-02  1.10836800e-01
   6.04210462e-02 -1.56441772e+00 -1.51077603e+00 -1.48012173e+00
  -1.49268074e+00 -1.49689234e+00  1.50332830e+00  1.55752198e-01
   1.39361839e-01  1.08134867e-01  1.11314491e-01  1.76922734e-01
   1.53289472e-01  1.25092522e-01  1.24754797e-01  1.88634794e-01
   1.49300132e-01 -1.43686702e+00  1.20292836e-01 -1.46654512e+00
   1.73350308e-01 -2.93794542e-01  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00 -5.99851219e-01]
 [-8.41275774e+00  0.00000000e+00 -3.64028377e-02 -3.46648568e-01
   2.86793494e-01 -2.56704214e-01  2.80901319e-01  5.69357635e-03
   2.17058093e-03  1.02967928e-02  4.32677180e-02  6.95486832e-03
   4.35850417e-02  3.11737867e+00  2.87745626e+00  2.88978918e+00
   3.01641479e+00  2.91055514e+00 -3.00051036e+00  4.41951431e-02
  -8.91318392e-03 -1.4498

**(c) Realizar predicciones y encontrar efectividad** Realizamos predicciones en el conjunto de prueba utilizando los parámetros theta entrenados y evaluamos la precisión del modelo.

In [47]:
predicciones = predictOneVsAll(theta, X_test)
precision = np.mean(predicciones.ravel() == y_test.ravel()) * 100
print("Precisión de las nuevas predicciones en el conjunto de prueba: {:.2f}%".format(precision))

Precisión de las nuevas predicciones en el conjunto de prueba: 56.64%


Y este modelo de otra forma:

In [48]:
from sklearn.svm import SVC
classifier =  SVC()
classifier.fit(X_train, y_train)
print(classifier.score(X_test, y_test) * 100)

62.23454833597465
